In [ ]:
from numpy import fromfile, dtype, array, float32, uint16, uint32, uint64, int32
from os import path
import matplotlib.pylab as plt

import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [ ]:
filepath = "LAr_test_45Vbias_threshold8mV_dig2-usb25715_20250117154944-12.bin"
file = open(filepath, 'rb')
dt = dtype(float32)

wvfms_all = []
for i in range(3000):
    wvfms_event = []
    # read header for event
    event_number = fromfile(file, dtype=uint32, count=1)
    if len(event_number) == 0:
        print('i = ', i)
        break
    event_number = event_number[0]
    timestamp = fromfile(file, dtype=uint64, count=1)[0]
    num_samples = fromfile(file, dtype=uint32, count=1)[0]
    sampling_period = fromfile(file, dtype=uint64, count=1)[0]
    channels = fromfile(file, dtype=uint32, count=1)[0]
    #print('event number: ', event_number)
    #print('timestamp: ', timestamp)
    #print('num_samples: ', num_samples)
    #print('sampling period: ', sampling_period)
    #print('number of channels: ', channels)
    for j in range(channels):
        channel = fromfile(file, dtype=uint16, count=1)[0]
        #print('channel: ', channel)
        trace = fromfile(file, dtype=dt, count=num_samples)
        wvfms_event.append(trace)

    wvfms_all.append(wvfms_event)

file.close()

In [ ]:
len(wvfms_all)

In [ ]:
len(wvfms_all[0][0])

In [ ]:
12500*6*452

In [ ]:
def filter_data(data, cutoff_freq, order):
    sampling_freq = 125e6
    normalized_cutoff_freq = cutoff_freq / (sampling_freq / 2)
    b, a = butter(order, normalized_cutoff_freq, btype='lowpass', analog=False)
    filtered_waveform = filtfilt(b, a, data)
    return filtered_waveform

In [ ]:
amplitudes = []
for ID in range(len(wvfms_all)):
    for i in range(6):
        if i == 1:
            continue
        wvfm = filter_data(wvfms_all[ID][i], 2e6, 2)
        amplitudes.append(np.max(wvfm))
        if np.max(wvfm) > 200:
            print('ID = ', ID)
        #plt.plot(wvfms_all[11][i])
        #plt.plot(np.arange(len(wvfms_all[ID][i]))*0.008, filter_data(wvfms_all[ID][i], 10e6, 2))
        #plt.xlim(10, 30)
    

#plt.xlabel('Time [usec]')
#plt.show()

In [ ]:
amplitudes

In [ ]:
plt.hist(amplitudes, range=(0,4), bins=20)
plt.yscale('log')
#plt.xlim(0, 100)
plt.show()


In [ ]:
plt.figure()
ID = 0
for i in range(6):
    
    #plt.plot(wvfms_all[11][i])
    plt.plot(np.arange(len(wvfms_all[ID][i]))*0.008, filter_data(wvfms_all[ID][i], 2e6, 2), label=f'Channel {i}')
    plt.xlim(10, 30)
    plt.legend()
    
plt.xlabel('Time [usec]')
plt.show()